# Multiple Linear Regression
## Car Price Case Study

#### Problem Statement:

This assignment submission attempts to build a multiple linear regression model for the prediction of car prices. 

A Chinese automobile company __Geely Auto__ aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts. 

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. 
The company wants to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. 

The company wants to know:
- Which variables are significant in predicting the price of a car
- How well those variables describe the price of a car

Based on various market surveys, the consulting firm has gathered a large dataset of different types of cars across the Americal market. 

__Business Goal__ 

This notebook will model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market. 

 

## Step 1: Reading and Understanding the Data



In [ ]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm



In [ ]:

#defining a function for calculating VIFs for passed in DF
def build_VIF(X):
    vif = pd.DataFrame()
    
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    print(vif)



In [ ]:
cars = pd.read_csv("CarPrice_Assignment.csv")

In [ ]:
# Add the Car Company column by extracting it from the CarName column and drop the CarName column now
cars['carco'] = cars['CarName'].str.split(n=1, expand=True)[0]
cars.drop(columns=['CarName'],inplace=True)

In [ ]:
# Check the head of the dataset
cars.head()

Inspect the various aspects of the cars dataframe

In [ ]:
cars.shape

In [ ]:
cars.info()

In [ ]:
cars.describe()

## Step 2: Visualising and Understanding Data


#### Visualising Numeric Variables
Pairplot of all the numeric variables

In [ ]:

sns.pairplot(cars, height=5)
plt.show()

#### Based on the pairplot above 

1. *wheelbase* seems to have a discernible pattern of relationship with *price*
2. *carwidth*, *carheight* & *curbweight* seem to have a discernible pattern of positive correlation with *price*.
    These 3 variable also seem to be strongly correlated amongst themselves.
3. *boreratio* & *stroke* seem to have a discernible pattern of positive correlation with *price*.
4. *citympg* & *highwaympg* seem to have a discernible negative correlation with *price* and a positive correlation between themselves.


#### Visualising Categorical Variables

Boxplot for categorical variables.

In [ ]:
plt.figure(figsize=(20, 12))

plt.subplot(4,3,1)
sns.boxplot(x = 'carco', y = 'price', data = cars)
plt.subplot(4,3,2)
sns.boxplot(x = 'fueltype', y = 'price', data = cars)
plt.subplot(4,3,3)
sns.boxplot(x = 'aspiration', y = 'price', data = cars)
plt.subplot(4,3,4)
sns.boxplot(x = 'doornumber', y = 'price', data = cars)
plt.subplot(4,3,5)
sns.boxplot(x = 'carbody', y = 'price', data = cars)
plt.subplot(4,3,6)
sns.boxplot(x = 'drivewheel', y = 'price', data = cars)
plt.subplot(4,3,7)
sns.boxplot(x = 'enginelocation', y = 'price', data = cars)
plt.subplot(4,3,8)
sns.boxplot(x = 'enginetype', y = 'price', data = cars)
plt.subplot(4,3,9)
sns.boxplot(x = 'cylindernumber', y = 'price', data = cars)
plt.subplot(4,3,10)
sns.boxplot(x = 'fuelsystem', y = 'price', data = cars)
plt.subplot(4,3,11)
sns.boxplot(x = 'symboling', y = 'price', data = cars)

plt.show()

In [ ]:
cars.carco.value_counts()


## Step 3: Cleansing the Data

In [ ]:
#
# Data Cleansing
#

# Car names have misspelt names
#

names_conv_dict = {"carco": 
                  {"toyouta": "toyota", \
                   "vw": "volkswagen", "maxda":"mazda", "porcshce":"porsche", "vokswagen":"volkswagen" }
                  }
cars.replace(names_conv_dict, inplace=True)
cars.carco.value_counts()

## Step 4 : Data Preparation 

In [ ]:
#
# Dropping insignificant columns
#

# From the box plot above of category variales, it appears that both *fueltype* and "doornumber" seem to be
# neutral to the impact on price. Hence dropping them.
#
cars.drop(columns=['fueltype','doornumber', 'car_ID'], inplace=True)



In [ ]:
#
# Convert category variables to dummy variables
#

# Convert doornumber & cylindernumber
nums_conv_dict = {"cylindernumber": {"four": 4, "six": 6, "five": 5, "eight": 8,\
                                     "two": 2, "twelve": 12, "three":3 }
                  }

cars.replace(nums_conv_dict, inplace=True)



In [ ]:
# Visualizing cylindernumber as boxplot after conversion for better readability.

plt.figure(figsize=(10, 8))
sns.boxplot(x = 'cylindernumber', y = 'price', data = cars)
#plt.subplot(4,3,10)
plt.show()

In [ ]:
cars.loc[cars.cylindernumber<=3]

###### There are only a few rows (5 out of 143) with cylindernumber 2 or 3. Barring these, there seem to be a highly positive correlation with price.


In [ ]:
# get dummies for car company names
#carco = pd.get_dummies(cars['carco'])
carco = pd.get_dummies(cars['carco'], drop_first=True)
carco.head()

In [ ]:
cars = pd.concat([cars, carco], axis=1)
cars.drop(columns=['carco'], inplace=True)
cars.head()

In [ ]:
asp = pd.get_dummies(cars['aspiration'],drop_first=True)

cars = pd.concat([cars, asp], axis=1)
cars.drop(columns=['aspiration'], inplace=True)
cars.head()

In [ ]:
# get dummies for carbody
carbody = pd.get_dummies(cars['carbody'], drop_first=True)

cars = pd.concat([cars, carbody], axis=1)
cars.drop(columns=['carbody'], inplace=True)
cars.head()

In [ ]:
# get dummies for enginelocation
eloc = pd.get_dummies(cars['enginelocation'], drop_first=True)
cars = pd.concat([cars, eloc], axis=1)
cars.drop(columns=['enginelocation'], inplace=True)
cars.head()

In [ ]:
# convert engine type

#
# First, let's treat all "ohc" type engines the same
#
etype_conv_dict =  {"enginetype": {"ohcf":"ohc", "ohcv":"ohc", "dohc":"ohc", "dohcv":"ohc"}}
cars.replace(etype_conv_dict, inplace=True)
cars.enginetype.head(200)

# now get dummies for enginetype
etype = pd.get_dummies(cars['enginetype'],drop_first=True)
etype.head()

cars = pd.concat([cars, etype], axis=1)
cars.drop(columns=['enginetype'], inplace=True)
cars.head()



In [ ]:
# now get dummies for drivewheel 
drwheel = pd.get_dummies(cars['drivewheel'],drop_first=True)
drwheel.head()

cars = pd.concat([cars, drwheel], axis=1)
cars.drop(columns=['drivewheel'], inplace=True)
cars.head()


In [ ]:
# now get dummies for fuelsystem 
fsys = pd.get_dummies(cars['fuelsystem'],drop_first=True)
fsys.head()

cars = pd.concat([cars, fsys], axis=1)
cars.drop(columns=['fuelsystem'], inplace=True)
cars.head()



## Step 5: Splitting the Data into Training and Testing Sets


In [ ]:
from sklearn.model_selection import train_test_split

# Specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
cars_train, cars_test = train_test_split(cars, train_size = 0.7, test_size = 0.3, random_state = 100)

### Rescaling the Features 

Using *Min-Max scaling* method. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Apply scaler() to relevant columns 
#num_vars = ['wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'horsepower', \
#            'peakrpm', 'citympg', 'highwaympg', 'price', 'compressionratio','symboling', 'cylindernumber' \
#            ]
num_vars = cars_train.columns

cars_train[num_vars] = scaler.fit_transform(cars_train[num_vars])


In [ ]:
cars_train.head()

In [ ]:
cars_train.describe()

In [ ]:
cars_train.info()

### Dividing into X and Y sets for the model building

In [ ]:
y_train = cars_train.pop('price')
X_train = cars_train

## Step 6: Building a linear model using RFE


In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output number of the variable equal to 10 since,
# based on the heatmap above, there seem to be 10 variables that have strong correlation with price.
#
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 10)             # running RFE
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]
col

In [ ]:
X_train.columns[~rfe.support_]

### Now Build Model to get stats values based on the top 10 variables from RFE process above.



In [ ]:
# Assign all the feature variables to X_train_rfe
X_train_rfe = X_train[col]

In [ ]:
# Build a linear model, print stats summary & VIF values

import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train_rfe)

lm = sm.OLS(y_train, X_train_lm).fit()

print(lm.summary())
build_VIF(X_train_rfe)

##### All  p-values are looking good. VIF values are very high. Some variables such as cylindernumber have a negative coefficient which is at odds with their individual correlation with price. 

#### Build the FIRST iteration of model over RFE model by dropping *cylindernumber* since by itself it has a positive correlation with price and yet in this model it shows a negative coefficient. It seems like an unreliable variable for the model. 

In [ ]:
X_train_1 = X_train_rfe.drop(["cylindernumber"], axis = 1)

# Build first revised fitted model
X_train_lm = sm.add_constant(X_train_1)
lm_1 = sm.OLS(y_train, X_train_lm).fit()

# Print the summary of the model
print(lm_1.summary())

# Calculate the VIFs again for the new model (after dropping feature)
build_VIF(X_train_1)


#### Build SECOND iteration of model over RFE model by dropping *stroke* due to high p-value.

In [ ]:
X_train_2 = X_train_1.drop(["stroke"], axis = 1)

# Build the second revised fitted model
X_train_lm = sm.add_constant(X_train_2)
lm_2 = sm.OLS(y_train, X_train_lm).fit()

# Print the summary of the model
print(lm_2.summary())

# Calculate the VIFs again for the new model (after dropping variable)
build_VIF(X_train_2)

#### Build THIRD iteration of model over RFE model by dropping *boreratio* for the same reason as *cylindernumber* (-ve coef but +ve correlation by itself with price.

In [ ]:
X_train_3 = X_train_2.drop(["boreratio"], axis = 1)

# Build a third fitted model
X_train_lm = sm.add_constant(X_train_3)
lm_3 = sm.OLS(y_train, X_train_lm).fit()

# Print the summary of the model
print(lm_3.summary())

# Calculate the VIFs again for the new model 
build_VIF(X_train_3)


####  Build a FOURTH iteration of the model dropping *curbweight* due to high VIF

In [ ]:
X_train_4 = X_train_3.drop(["curbweight"], axis = 1)

# Build a fourth fitted model
X_train_lm = sm.add_constant(X_train_4)
lm_4 = sm.OLS(y_train, X_train_lm).fit()

# Print the summary of the model
print(lm_4.summary())

# Calculate the VIFs again for the new model 
build_VIF(X_train_4)


#### Build the FIFTH iteration, dropping  *porsche*  due to high p-value

In [ ]:
X_train_5 = X_train_4.drop(["porsche"], axis = 1)

# Build a fifth fitted model
X_train_lm = sm.add_constant(X_train_5)
lm_5 = sm.OLS(y_train, X_train_lm).fit()

# Print the summary of the model
print(lm_5.summary())

# Calculate the VIFs again for the new model 
build_VIF(X_train_5)


#### Build the SIXTH iteration, dropping  *carwidth* due to high VIF (instead of *enginesize* since it is better to  to retain that one for  interpretability)

In [ ]:
X_train_6 = X_train_5.drop(["carwidth"], axis = 1)

# Build a fifth fitted model
X_train_lm = sm.add_constant(X_train_6)
lm_6 = sm.OLS(y_train, X_train_lm).fit()

# Print the summary of the model
print(lm_6.summary())

# Calculate the VIFs again for the new model 
build_VIF(X_train_6)


#### Build the SEVENTH iteration, dropping  *peugeot* due to high p-Value

In [ ]:
X_train_7 = X_train_6.drop(["peugeot"], axis = 1)

# Build a fifth fitted model
X_train_lm = sm.add_constant(X_train_7)
lm_7 = sm.OLS(y_train, X_train_lm).fit()

# Print the summary of the model
print(lm_7.summary())

# Calculate the VIFs again for the new model 
build_VIF(X_train_7)


### A Decent Model now with good p-Values, a moderate Adj R-squared value of 79% & low VIF values (all below 2).

## Step 7: Residual Analysis of the train data

Now to check if the error terms are also normally distributed (which is a major assumption of linear regression), plot the histogram of the error terms and see what it looks like.

In [ ]:
#lm = LinearRegression()
lm = sm.OLS(y_train, X_train_lm).fit()
y_train_price = lm.predict(X_train_lm)


In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_price), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

##### The error histogram seems to have approximate normal distribution. However, two concerns
1. Mean is centred below zero
2. The right tail is a bit long.

No better option at this stage.

## Step 8: Making Predictions Using the Final Model

Now that we have fitted the model and checked the normality of error terms,  
make predictions using the final, i.e. third model.

#### Applying the scaling on the test sets

In [ ]:
scaler = MinMaxScaler()

# Apply scaler() to relevant columns 
#num_vars = ['wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'horsepower', \
#            'peakrpm', 'citympg', 'highwaympg', 'compressionratio','symboling']

num_vars = cars_test.columns

cars_test[num_vars] = scaler.fit_transform(cars_test[num_vars])



In [ ]:
cars_test.describe()

#### Dividing into X_test and y_test

In [ ]:
y_test = cars_test.pop('price')
X_test = cars_test

In [ ]:
# Adding constant variable to test dataframe
X_test = sm.add_constant(X_test)

In [ ]:
# Creating X_test dataframe 

cols = X_train_7.columns
X_test = X_test[cols]
X_test.describe()

In [ ]:
# Making predictions using the third  model

lm = sm.OLS(y_test, X_test).fit()
y_pred = lm.predict(X_test)

## Step 9: Model Evaluation

Plot the graph for actual versus predicted values.

In [ ]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
plt.scatter(y_test, y_pred)
fig.suptitle('y_test vs y_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_pred', fontsize = 16)      

So, We can see that the equation of our best fitted line is:

$ price = 1.1  \times  enginesize + 1.1  \times  bmw +  rear$

However, there is a bit of variance showing in the lower values. 

## NOW, Evaluate an ALTERNATE model, built using RFE without further iterations.

This model has a very high Adj R-Squared of 90% and good p-values.
Only anamoly is high VIFs for some variables and -ve coef for some features which by themselves +vely correlated to price.

$ price = 1.2 \times enginesize + 0.43  \times  carwidth + 0.35 \times curbweight + + 0.27 \times rear + 0.24 \times bmw + 0.23 \times peugeot  - 0.91  \times cylindernumber - 0.37 \times boreratio - 0.3 \times stroke  - 0.12 \times peugeot $


### Step 7A: Residual Analysis of the train data 

Check if the error terms are also normally distributed (which is a major assumption of linear regression), plot the histogram of the error terms and see what it looks like.

In [ ]:
cars_train, cars_test = train_test_split(cars, train_size = 0.7, test_size = 0.3, random_state = 100)

#lm = LinearRegression()
X_train_lm = X_train_rfe
X_train_lm = sm.add_constant(X_train_rfe)
lm = sm.OLS(y_train, X_train_lm).fit()
y_train_price = lm.predict(X_train_lm)


In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_price), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

##### The error histogram seems to have approximate normal distribution and seems to be in better shape than the one evaluated for the previous model. The right tail is more gradual compared to the previous model. The Mean is still centred slightly below zero.


## Step 8A: Making Predictions Using the Final Model

Now that we have fitted the model and checked the normality of error terms,  
make predictions using the final, i.e. third model.

#### Applying the scaling on the test sets

In [ ]:
scaler = MinMaxScaler()

# Apply scaler() to relevant columns 
#num_vars = ['wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'horsepower', \
#            'peakrpm', 'citympg', 'highwaympg', 'compressionratio','symboling']

num_vars = cars_test.columns

cars_test[num_vars] = scaler.fit_transform(cars_test[num_vars])



In [ ]:
cars_test.describe()

#### Dividing into X_test and y_test

In [ ]:
y_test = cars_test.pop('price')

X_test = cars_test

In [ ]:
# Adding constant variable to test dataframe
X_test = sm.add_constant(X_test)

In [ ]:
# Creating X_test dataframe 

cols = X_train_1.columns
X_test = X_test[cols]
X_test.describe()

In [ ]:
# Making predictions using the third  model

lm = sm.OLS(y_test, X_test).fit()
y_pred = lm.predict(X_test)

## Step 9A: Model Evaluation

Plot the graph for actual versus predicted values.

In [ ]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
plt.scatter(y_test, y_pred)
fig.suptitle('y_test vs y_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_pred', fontsize = 16)      


So, We can see that the equation of our best fitted line for this alternate model is


$ price = 1.2 \times enginesize + 0.43  \times  carwidth + 0.35 \times curbweight + + 0.27 \times rear + 0.24 \times bmw + 0.23 \times peugeot  - 0.91  \times cylindernumber - 0.37 \times boreratio - 0.3 \times stroke  - 0.12 \times peugeot $

## Conclusion / Intepretations :

1. The final fitted y-test & y-pred graphs between the two models (X_train_7 & X_Train_RFE) are not very different.

2. Since the X_train_7 models has fewer variables and easier to interpret, we shall go with that model which has the following equation
$ price = 1.1  \times  enginesize + 1.1  \times  bmw +  rear$

This tells us that the price of the cars is most positively affected by the following features, in that order
    1. Engine size,
    2. Whether it is a BMW & 
    3. Whether it has a rear engine
    
Interestingly, as it happens, BMW does not make rear-engine cars, which means features 2 & 3 are mutually exclusive. 

Additional interpretations of this:
1. A non-BMW car will have a higher price than a BMW only if it's engine size is larger than the BMW car.
2. A BMW car will always have the best price as long as the cars being compared are of smaller size.